In [60]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

In [52]:
train_features = pd.read_csv('train/X_train.txt',delimiter="\s+",header=None)
train_labels = pd.read_csv('train/y_train.txt', delimiter = "\s+",header=None)
test_features = pd.read_csv('test/X_test.txt', delimiter = "\s+",header=None)
test_labels = pd.read_csv('test/y_test.txt', delimiter = "\s+",header=None)

In [57]:
print(train_features.shape,train_labels.shape)
print(test_features.shape,test_labels.shape)

(7352, 561) (7352, 1)
(2947, 561) (2947, 1)


In [58]:
train_features.head(5)

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [59]:
corrMatrix = train_features.corr()

In [62]:
#sn.heatmap(corrMatrix, annot=True)
#plt.show()
print(corrMatrix)

          0         1         2         3         4         5         6    \
0    1.000000  0.148061 -0.256952  0.000619 -0.021903 -0.044617  0.006290   
1    0.148061  1.000000 -0.078769 -0.045160 -0.044920 -0.049746 -0.044180   
2   -0.256952 -0.078769  1.000000 -0.020217 -0.016641 -0.008410 -0.018747   
3    0.000619 -0.045160 -0.020217  1.000000  0.927461  0.851668  0.998632   
4   -0.021903 -0.044920 -0.016641  0.927461  1.000000  0.895510  0.922803   
..        ...       ...       ...       ...       ...       ...       ...   
556  0.037444  0.017967 -0.063609  0.018985 -0.008507 -0.018429  0.019389   
557  0.028844  0.075679 -0.034037 -0.024810 -0.014592 -0.006471 -0.024951   
558 -0.035257 -0.005309  0.008587 -0.371653 -0.380531 -0.345011 -0.368191   
559  0.034371  0.001053 -0.015288  0.471065  0.523600  0.476006  0.466424   
560  0.028242 -0.013903 -0.022643  0.394825  0.433169  0.482828  0.390922   

          7         8         9    ...       551       552       553  \
0  

In [63]:
from sklearn.linear_model import Perceptron
model = Perceptron()
model.fit(train_features, train_labels)

/home/mihir/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Perceptron()

In [64]:
model.score(train_features, train_labels)

0.9808215451577802